#Object detection in a Video

We have divided the video into certain frames based on the video's framerate.
Predict on those particular frames and store them.
Make a video out of those frames.

In [1]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 274 kB 5.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=a47b16f262a4aae7e82cee4d3b7f274b6fc8c9020735b0855b0c9dd524a2121c
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 1.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Succe

In [2]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.3 MB 810 kB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 
     |████████████████████████████████| 130 kB 15.8 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
     |████████████████████████████████| 145 kB 39.8 MB/s 
     |████████████████████████████████| 843 kB 42.2 MB/s 
     |████████████████████████████████| 749 kB 31.2 MB/s 
     |████████████████████████████████| 112 kB 47.5 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60947 sha256=737459aa655199d3c6f6b23ca8ac5013b4af900d4d99627b3fc991a9ee0915f9
  Stored in directory: /root/.cache/pip/wheels/16/98/fc/252d62cab6263c719120e06b28f3378af59b52ce7a20e81852
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=fed87e16d26b823e6e78142ac442965767d12c73fac72856c649e8efa56914

In [1]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [2]:
# list of classes considered
classes = ['car', 'autorickshaw', 'motorbike', 'building', 'bridge', 'truck', 'person', 'bus', 'traffic light', 'traffic sign']
classes.sort()
print(classes)

['autorickshaw', 'bridge', 'building', 'bus', 'car', 'motorbike', 'person', 'traffic light', 'traffic sign', 'truck']


In [3]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

# config file for prediction
cfg = get_cfg()
# merge both the previous config file from training and the current one

cfg.merge_from_file("/content/drive/MyDrive/Coders++/new_model/output.yaml")

# add the model weights created from training
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/Coders++/new_model/model_final.pth")

# threshold for detecting objects
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
# creation of predictor object based on new config file with retrained neural network and configurations
predictor = DefaultPredictor(cfg)

In [9]:
# Get a frame from a video at a particular second
def getFrame(video, sec): 
    video.set(cv2.CAP_PROP_POS_MSEC, sec*1000) 
    has_frames,image = video.read() 
    if has_frames: 
      return has_frames, image 

In [33]:
import cv2
from google.colab.patches import cv2_imshow

# source video
video = cv2.VideoCapture("/content/drive/MyDrive/Coders++/videos/mumbai_traffic.mp4")

seconds = 0 #current second
frame_rate = 0.1 # FPS

fps = video.get(cv2.CAP_PROP_FPS)
print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))

MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).set(thing_classes = classes)

has_frames, image = getFrame(video, seconds)
size = (image.shape[1], image.shape[0])

# print(image.shape)

img_array = []

# extract the frames, predict and add them into an array
while has_frames:
  outputs = predictor(image)
  v = Visualizer(image[:, :, ::-1],
                 metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                 scale = 1,
                 instance_mode = ColorMode.SEGMENTATION)
  
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  # cv2_imshow(v.get_image()[:, :, ::-1])
  img_array.append(v.get_image()[:, :, ::-1])
  
  seconds += frame_rate
  try:
    has_frames, image = getFrame(video, seconds)
  except:
    break

# make a video out of those frames
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# destination video
out = cv2.VideoWriter("/content/mumbai_traffic.mp4", fourcc, 10, size)
 
for i in range(len(img_array)):
  out.write(img_array[i])
out.release()

Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): 29.97002997002997
